# CSC8002 Assignment 2

## subset_dataselection

In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import pymongo
from pymongo import MongoClient


import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter


import datetime

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [5]:
# Load Google Mobility Data
coviddf = pd.read_csv('C:\\Users\Justin\Desktop\Semester 3 2020\CSC8002 Big Data Management\Assignments\data.csv')

# Select all Australian data, including national level, state level and sub region level.
coviddfaus = coviddf.loc[coviddf['country_region_code'] == 'AU']

# Select national level data
coviddfausnat = coviddfaus[coviddfaus['sub_region_1'].isnull() & coviddfaus['sub_region_2'].isnull() 
                           & coviddfaus['iso_3166_2_code'].isnull()]

# Select items that have an iso code/select state level data only.
coviddfausnatiso = coviddfaus[coviddfaus['iso_3166_2_code'].notnull()]

# Drop unrelated columns to get rid of unnecessary data.
coviddfausnatisoclean = coviddfausnatiso.drop(columns=['country_region_code', 'country_region','sub_region_1',
                                                       'sub_region_2','metro_area','census_fips_code'])

# Create aggregate mean columns of essential and non-essential change from baseline

coviddfausnatisoclean['Mean_Essential'] = coviddfausnatisoclean[['grocery_and_pharmacy_percent_change_from_baseline',
                                                                 'residential_percent_change_from_baseline']].mean(axis=1)
coviddfausnatisoclean['Mean_Non_Essential'] = coviddfausnatisoclean[['retail_and_recreation_percent_change_from_baseline',
                                                                     'parks_percent_change_from_baseline',
                                                                     'transit_stations_percent_change_from_baseline',
                                                                     'workplaces_percent_change_from_baseline']].mean(axis=1)

# Drop redundant columns
coviddfausnatisoclean = coviddfausnatisoclean.drop(columns=['retail_and_recreation_percent_change_from_baseline',
                                    'grocery_and_pharmacy_percent_change_from_baseline',
                                    'parks_percent_change_from_baseline',
                                    'transit_stations_percent_change_from_baseline',
                                    'workplaces_percent_change_from_baseline',
                                    'residential_percent_change_from_baseline'])

coviddfausnatisoclean.to_csv('coviddfausnatisoclean.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### EDA

In [20]:
coviddfausnatisoclean

,iso_3166_2_code,date,Mean_Essential,Mean_Non_Essential
150462,AU-ACT,2020-02-15,2.0,11.25
150463,AU-ACT,2020-02-16,8.5,15.00
150464,AU-ACT,2020-02-17,3.0,13.50
150465,AU-ACT,2020-02-18,-1.5,5.75
150466,AU-ACT,2020-02-19,3.0,13.25
...,...,...,...,...
215111,AU-WA,2020-11-27,3.5,-8.25
215112,AU-WA,2020-11-28,3.5,-7.50
215113,AU-WA,2020-11-29,3.5,-14.25
215114,AU-WA,2020-11-30,5.0,-16.00


In [19]:
# Checking for duplicates with the same date
coviddfaus.loc[(coviddfaus['date'] == '2020-02-15')]

,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
150171,AU,Australia,NaN,NaN,NaN,NaN,NaN,2020-02-15,4.0,3.0,-2.0,3.0,3.0,0.0
150462,AU,Australia,Australian Capital Territory,NaN,NaN,AU-ACT,NaN,2020-02-15,7.0,5.0,28.0,4.0,6.0,-1.0
150753,AU,Australia,New South Wales,NaN,NaN,AU-NSW,NaN,2020-02-15,4.0,5.0,1.0,10.0,3.0,-1.0
151044,AU,Australia,New South Wales,Armidale Dumaresq Council,NaN,NaN,NaN,2020-02-15,7.0,NaN,NaN,NaN,NaN,NaN
151291,AU,Australia,New South Wales,Ballina Shire Council,NaN,NaN,NaN,2020-02-15,-5.0,-15.0,-17.0,-17.0,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224929,AU,Australia,Western Australia,Shire of Roebourne,NaN,NaN,NaN,2020-02-15,19.0,NaN,-2.0,NaN,-1.0,NaN
225671,AU,Australia,Western Australia,Town of Cambridge,NaN,NaN,NaN,2020-02-15,6.0,NaN,-3.0,NaN,NaN,NaN
225937,AU,Australia,Western Australia,Town of Claremont,NaN,NaN,NaN,2020-02-15,4.0,NaN,NaN,NaN,NaN,NaN
226103,AU,Australia,Western Australia,Town of Cottesloe,NaN,NaN,NaN,2020-02-15,NaN,NaN,-32.0,NaN,NaN,NaN


In [8]:
# National Data Shape
coviddfausnat.shape

(291, 14)

In [11]:
# State Data Shape
coviddfausnatisoclean.shape

(2328, 4)